In [1]:
#import dependecies
import pandas as pd
from pathlib import Path
import polars as pl
import re

In [2]:
df1= pd.read_csv("player_possession.csv")
df2 = pd.read_csv("player_shooting.csv")
df3 = pd.read_csv("player_passing.csv")
df4 = pd.read_csv("player_defense.csv")

In [3]:
# Right after import, drop repeated columns from data frames to prevent duplicating upon merge.
df4 = df4.drop(columns=['age', 'team','position','minutes_90s','birth_year'])

In [4]:
#Merge columns together using 'pd.merge'
merged_df = pd.merge(df1,df2,how='inner',left_on='player',right_on='player')
merged_df = pd.merge(merged_df,df3,how='inner',left_on='player',right_on='player')
merged_df = pd.merge(merged_df,df4,how='inner',left_on='player',right_on='player')

In [6]:
#Drop unnecessary columns
columns_to_drop = ['minutes_90s','touches_def_pen_area','touches_def_3rd','touches_mid_3rd','touches_att_3rd','touches_att_pen_area',
                  'touches_live_ball','dribbles','dribbles_completed_pct','miscontrols', 'dispossessed', 'passes_received','progressive_passes_received',
                  'shots_on_target_pct', 'shots_per90', 'shots_on_target_per90','shots_on_target_pct','shots_per90',
                  'goals_per_shot_on_target','average_shot_distance','passes_pct', 'passes_total_distance', 'passes_progressive_distance',
                  'passes_completed_short', 'passes_short', 'passes_pct_short','passes_completed_medium', 'passes_medium', 'passes_pct_medium','passes_completed_long', 'passes_long', 'passes_pct_long',
                  'xg_assist', 'pass_xa', 'xg_assist_net', 'assisted_shots','passes_into_final_third', 'passes_into_penalty_area','crosses_into_penalty_area', 'progressive_passes','tackles_def_3rd', 'tackles_mid_3rd', 'tackles_att_3rd'
                  ]
df_dropped = merged_df.drop(columns_to_drop, axis=1)



In [8]:
#Drop unnecessary columns
columns_2_drop = ['birth_year_y','minutes_90s_y','shots_free_kicks','pens_made','pens_att', 'xg', 'npxg', 'npxg_per_shot', 'xg_net', 'npxg_net',
                  'tackles', 'dribble_tackles', 'dribbles_vs',
                  'dribble_tackles_pct', 'dribbled_past', 'blocks', 'blocked_shots',
                  'blocked_passes', 'interceptions', 'tackles_interceptions','clearances', 'errors','birth_year',
                 'position','team', 'age','team', 'age','age_y','goals_per_shot','position_y','team_y','minutes_90s_x','birth_year_x']
data_df = df_dropped.drop(columns_2_drop, axis=1)


In [9]:
data_df.dtypes

player                 object
position_x             object
team_x                 object
age_x                  object
touches               float64
dribbles_completed    float64
goals                   int64
shots                   int64
shots_on_target         int64
passes_completed      float64
passes                float64
assists                 int64
tackles_won             int64
dtype: object

In [28]:
# Display Result
data_df

,player,position_x,team_x,age_x,touches,dribbles_completed,goals,shots,shots_on_target,passes_completed,passes,assists,tackles_won
0,Aaron Mooy,MF,Australia,32-094,255.0,2.0,0,1,0,170.0,217.0,0,6
1,Aaron Ramsey,MF,Wales,31-357,147.0,2.0,0,1,0,88.0,112.0,0,0
2,Abdelhamid Sabiri,MF,Morocco,26-020,86.0,0.0,0,3,0,45.0,58.0,1,1
3,Abdelkarim Hassan,DF,Qatar,29-112,193.0,1.0,0,6,0,122.0,161.0,0,3
4,Abderrazak Hamdallah,FW,Morocco,32-001,28.0,2.0,0,2,1,8.0,15.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,Ángel Di María,MF,Argentina,34-307,201.0,13.0,1,6,3,130.0,168.0,1,1
676,Ángelo Preciado,DF,Ecuador,24-303,162.0,0.0,0,4,0,94.0,129.0,1,5
677,Éder Militão,DF,Brazil,24-334,306.0,0.0,0,5,1,251.0,277.0,0,6
678,Óscar Duarte,DF,Costa Rica,33-198,132.0,0.0,0,0,0,68.0,98.0,0,2


In [31]:
# Create a Polars DataFrame
df = pl.DataFrame(data_df)
print(type(df))
df.head()

<class 'polars.dataframe.frame.DataFrame'>


player,position_x,team_x,age_x,touches,dribbles_completed,goals,shots,shots_on_target,passes_completed,passes,assists,tackles_won
str,str,str,str,f64,f64,i64,i64,i64,f64,f64,i64,i64
"""Aaron Mooy""","""MF""","""Australia""","""32-094""",255.0,2.0,0,1,0,170.0,217.0,0,6
"""Aaron Ramsey""","""MF""","""Wales""","""31-357""",147.0,2.0,0,1,0,88.0,112.0,0,0
"""Abdelhamid Sab…","""MF""","""Morocco""","""26-020""",86.0,0.0,0,3,0,45.0,58.0,1,1
"""Abdelkarim Has…","""DF""","""Qatar""","""29-112""",193.0,1.0,0,6,0,122.0,161.0,0,3
"""Abderrazak Ham…","""FW""","""Morocco""","""32-001""",28.0,2.0,0,2,1,8.0,15.0,0,0


In [12]:
# Rename the columns
new_column_names = {
    'position_x': 'position',
    'age_x': 'age',
    'team_x': 'team'
}

df_renamed = df.rename(new_column_names)

# Display the result
df_renamed.head()

player,position,team,age,touches,dribbles_completed,goals,shots,shots_on_target,passes_completed,passes,assists,tackles_won
str,str,str,str,f64,f64,i64,i64,i64,f64,f64,i64,i64
"""Aaron Mooy""","""MF""","""Australia""","""32-094""",255.0,2.0,0,1,0,170.0,217.0,0,6
"""Aaron Ramsey""","""MF""","""Wales""","""31-357""",147.0,2.0,0,1,0,88.0,112.0,0,0
"""Abdelhamid Sab…","""MF""","""Morocco""","""26-020""",86.0,0.0,0,3,0,45.0,58.0,1,1
"""Abdelkarim Has…","""DF""","""Qatar""","""29-112""",193.0,1.0,0,6,0,122.0,161.0,0,3
"""Abderrazak Ham…","""FW""","""Morocco""","""32-001""",28.0,2.0,0,2,1,8.0,15.0,0,0


In [13]:
# Drop unnecesary values from columns

df_5 = df_renamed.with_columns(pl.col('age').map_elements(lambda value:
                                                     value.split('-')[0]))
df_5.head()

player,position,team,age,touches,dribbles_completed,goals,shots,shots_on_target,passes_completed,passes,assists,tackles_won
str,str,str,str,f64,f64,i64,i64,i64,f64,f64,i64,i64
"""Aaron Mooy""","""MF""","""Australia""","""32""",255.0,2.0,0,1,0,170.0,217.0,0,6
"""Aaron Ramsey""","""MF""","""Wales""","""31""",147.0,2.0,0,1,0,88.0,112.0,0,0
"""Abdelhamid Sab…","""MF""","""Morocco""","""26""",86.0,0.0,0,3,0,45.0,58.0,1,1
"""Abdelkarim Has…","""DF""","""Qatar""","""29""",193.0,1.0,0,6,0,122.0,161.0,0,3
"""Abderrazak Ham…","""FW""","""Morocco""","""32""",28.0,2.0,0,2,1,8.0,15.0,0,0


In [14]:
# Casting data to their corrisponding data type

new_columns = df_5.with_columns(
    pl.col("age").cast(pl.Int64),
    pl.col("touches").cast(pl.Int64),
    pl.col("dribbles_completed").cast(pl.Int64),
    pl.col("passes_completed").cast(pl.Int64),
    pl.col("passes").cast(pl.Int64)
)
new_columns.tail()

player,position,team,age,touches,dribbles_completed,goals,shots,shots_on_target,passes_completed,passes,assists,tackles_won
str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""Ángel Di María…","""MF""","""Argentina""",34,201,13,1,6,3,130,168,1,1
"""Ángelo Preciad…","""DF""","""Ecuador""",24,162,0,0,4,0,94,129,1,5
"""Éder Militão""","""DF""","""Brazil""",24,306,0,0,5,1,251,277,0,6
"""Óscar Duarte""","""DF""","""Costa Rica""",33,132,0,0,0,0,68,98,0,2
"""İlkay Gündoğan…","""MF""","""Germany""",32,187,2,1,5,2,150,168,0,1


In [15]:
#Drop any null values in the DataFrame
new_columns = new_columns.drop_nulls()

In [16]:
#sort values depending on ammount of goals
final_df = new_columns.sort(["goals"], descending=True)

In [18]:
#Display final results
final_df

player,position,team,age,touches,dribbles_completed,goals,shots,shots_on_target,passes_completed,passes,assists,tackles_won
str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""Kylian Mbappé""","""FW""","""France""",23,325,25,8,29,11,173,229,2,0
"""Lionel Messi""","""FW""","""Argentina""",35,460,15,7,27,13,301,372,3,3
"""Julián Álvarez…","""FW""","""Argentina""",22,174,0,4,11,8,93,121,0,2
"""Olivier Giroud…","""FW""","""France""",36,111,0,4,16,6,48,74,0,0
"""Bukayo Saka""","""FW""","""England""",21,147,2,3,7,5,84,108,0,4
"""Cody Gakpo""","""FW""","""Netherlands""",23,205,5,3,5,3,93,148,0,2
"""Enner Valencia…","""FW""","""Ecuador""",33,98,2,3,8,3,40,57,0,0
"""Gonçalo Ramos""","""FW""","""Portugal""",21,55,0,3,7,5,25,31,1,1
"""Marcus Rashfor…","""FW""","""England""",25,92,5,3,11,6,49,66,0,0


In [39]:
#Create DataFame
final_df = pl.DataFrame(final_df)
#Save DataFrame as a csv file
final_df.write_csv('FIFA_stats_cleaned.csv')

